In [ ]:
from YouTubeDownload import YouTubeDownload
from MusicSeparation import MusicSeparation
from pytubefix import YouTube
import pytubefix
import os

url = input()

def download_youtube_audio(video_url, mode = "audio"):
    if mode == "audio" :
        try:
            yt = YouTube(video_url)
            title = YouTubeDownload.sanitize_filename(yt.title)
            audio_stream = yt.streams.get_audio_only()

            os.makedirs("download", exist_ok=True)
            os.path.join("download", f"{title}.m4a")
            m4a_path = audio_stream.download(output_path="download", filename=f"{title}.m4a")

            return f'"{m4a_path}"'
        except Exception as e:
            return f"錯誤：{e}"
    elif mode == "playlist":
        for audio in pytubefix.Playlist(video_url):
            print(audio)
            temp = []
            try:
                yt = YouTube(audio)
                title = YouTubeDownload.sanitize_filename(yt.title)
                audio_stream = yt.streams.get_audio_only()

                os.makedirs("download", exist_ok=True)
                os.path.join("download", f"{title}.m4a")
                temp.append(audio_stream.download(output_path="download", filename=f"{title}.m4a"))
                
            except Exception as e:
                temp.append(f"錯誤：{e}")
        return temp
#download_youtube_audio(url, mode="playlist")
audio = YouTubeDownload.download_youtube_audio(url)
audio


In [ ]:
temp = []
for str1 in audio:
    temp.append(MusicSeparation.run_separation(str1))
temp

In [ ]:
import os
import csv
import subprocess
import glob
import tempfile
import shutil
from montreal_forced_aligner.command_line import align_corpus_cli, g2p_cli

def align_song_mfa(audio_path, output_dir, acoustic_model_path, g2p_model, dict_file):
    """
    使用 MFA 對齊歌曲 + 歌詞，並切割成一句一句音檔 + CSV
    """
    os.makedirs(output_dir, exist_ok=True)

    # 建立臨時 corpus 資料夾
    corpus_dir = os.path.join(output_dir, "corpus")
    os.makedirs(corpus_dir, exist_ok=True)

    # 複製音檔
    song_copy = os.path.join(corpus_dir, os.path.basename(audio_path))
    if not os.path.exists(song_copy):
        shutil.copy(audio_path, song_copy)

    # 嘗試找同名 txt 歌詞檔，並複製成 .lab
    base_name = os.path.splitext(os.path.basename(audio_path))[0]
    lyrics_txt = os.path.join(os.path.dirname(audio_path), f"{base_name}.txt")
    if os.path.exists(lyrics_txt):
        lab_copy = os.path.join(corpus_dir, f"{base_name}.lab")
        shutil.copy(lyrics_txt, lab_copy)
        print(f"📝 已複製歌詞 {lyrics_txt} → {lab_copy}")
    else:
        print(f"⚠️ 找不到歌詞檔 {lyrics_txt}，這首歌可能無法對齊")

    # MFA 輸出 TextGrid
    textgrid_dir = os.path.join(output_dir, "textgrid")
    os.makedirs(textgrid_dir, exist_ok=True)

    args = [
        corpus_dir,        # 改成單首歌的 corpus 資料夾
        dict_file,
        acoustic_model_path,
        textgrid_dir,
        "--clean",
        #"--ignore_oovs",   #忽略OOV詞
        "--beam", "100",
        "--retry_beam", "400"
    ]
    print(f"🎤 正在對齊 {os.path.basename(audio_path)} ...")
    try:
        align_corpus_cli(args)
    except SystemExit:
        pass

    # 讀取 TextGrid 並切割音檔
    import textgrid
    tg_path = glob.glob(os.path.join(textgrid_dir, "*.TextGrid"))[0]
    tg = textgrid.TextGrid.fromFile(tg_path)

    csv_path = os.path.join(output_dir, "metadata.csv")
    with open(csv_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["file", "text"])
        tier = tg.tiers[0]  # 只取第一層 tier

        for i, interval in enumerate(tier.intervals):
            text = interval.mark.strip()
            if not text:
                continue

            begin = interval.minTime
            end = interval.maxTime
            out_wav = os.path.join(output_dir, f"line_{i:04d}.wav")

            command = [
                "ffmpeg", "-y", "-i", audio_path,
                "-ss", str(begin), "-to", str(end),
                "-ar", "16000", "-ac", "1",
                out_wav
            ]
            subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            writer.writerow([out_wav, text])

    return csv_path


def process_folder_mfa(input_dir, acoustic_model_path, g2p_model, dict_file, output_root="processed_songs", merged_csv="metadata_all.csv"):
    """
    批次處理資料夾內所有歌曲，用 MFA 對齊
    """
    os.makedirs(output_root, exist_ok=True)
    all_rows = []

    audio_files = glob.glob(os.path.join(input_dir, "*.wav")) + \
                  glob.glob(os.path.join(input_dir, "*.mp3")) + \
                  glob.glob(os.path.join(input_dir, "*.flac"))

    for audio_path in audio_files:
        base_name = os.path.splitext(os.path.basename(audio_path))[0]
        song_output = os.path.join(output_root, base_name)
        csv_path = align_song_mfa(audio_path, song_output, acoustic_model_path, g2p_model, dict_file)

        if csv_path:
            with open(csv_path, "r", encoding="utf-8") as f:
                reader = csv.reader(f)
                next(reader)
                for row in reader:
                    all_rows.append(row)

    merged_path = os.path.join(output_root, merged_csv)
    with open(merged_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["file", "text"])
        writer.writerows(all_rows)

    print(f"\n✅ 所有歌曲已處理完成！")
    print(f"👉 輸出資料夾: {output_root}")
    print(f"👉 合併 metadata: {merged_path}")

In [2]:
"""mfa model list acoustic
['english_mfa', 'japanese_mfa', 'mandarin_mfa']

mfa model list dictionary
['english_mfa', 'japanese_mfa', 'mandarin_pinyin', 'mandarin_taiwan_mfa']

mfa model list g2p
['english_us_mfa', 'japanese_mfa', 'mandarin_taiwan_mfa', 'mandarin_taiwan_pinyin_mfa']
"""
import os

home_dir = os.path.expanduser("~")
acoustic_model_path = os.path.join(home_dir, "Documents", "MFA", "pretrained_models", "acoustic")
g2p_model = os.path.join(home_dir, "Documents", "MFA", "pretrained_models", "g2p")
dict_file = os.path.join(home_dir, "Documents", "MFA", "pretrained_models", "dictionary")

acoustic_model_path += "\\mandarin_mfa.zip"
dict_file += "\\mandarin_taiwan_mfa.dict"
g2p_model += "\\mandarin_taiwan_mfa.zip"


input_dir = "song_dataset"

print(input_dir, acoustic_model_path, g2p_model)

try:
    process_folder_mfa(input_dir, acoustic_model_path, g2p_model, dict_file)
except SystemExit:  #避免exit()回傳錯誤
    pass 

song_dataset C:\Users\a0987\Documents\MFA\pretrained_models\acoustic\mandarin_mfa.zip C:\Users\a0987\Documents\MFA\pretrained_models\g2p\mandarin_taiwan_mfa.zip
📝 已複製歌詞 song_dataset\周杰倫 Jay Chou (特別演出_ 派偉俊)【告白氣球 Love Confession】Official MV-vocals.txt → processed_songs\周杰倫 Jay Chou (特別演出_ 派偉俊)【告白氣球 Love Confession】Official MV-vocals\corpus\周杰倫 Jay Chou (特別演出_ 派偉俊)【告白氣球 Love Confession】Official MV-vocals.lab
🎤 正在對齊 周杰倫 Jay Chou (特別演出_ 派偉俊)【告白氣球 Love Confession】Official MV-vocals.wav ...


 INFO     Setting up corpus information...

 INFO     Loading corpus from source files...

Output()

c:\Users\a0987\anaconda3\envs\aligner\Lib\site-packages\montreal_forced_aligner\corpus\acoustic_corpus.py:892: TqdmExperimentalWarning: rich is experimental/alpha
  with self.session() as session, tqdm(total=100, disable=config.QUIET) as pbar:


...

 INFO     Exporting alignment TextGrids to processed_songs\周杰倫 Jay Chou【蘭亭序 Lan-Ting-Xu】-Official Music    
          Video-vocals\textgrid...

Output()

 INFO     Finished exporting TextGrids to processed_songs\周杰倫 Jay Chou【蘭亭序 Lan-Ting-Xu】-Official Music     
          Video-vocals\textgrid!

 INFO     Done! Everything took 108.056 seconds


✅ 所有歌曲已處理完成！
👉 輸出資料夾: processed_songs
👉 合併 metadata: processed_songs\metadata_all.csv
